In [1]:
"""Basic demonstration of sweeps and metrics operation."""

# %%
# Imports, etc.

import numpy as np
from functools import partial
import torch


from transformer_lens import HookedTransformer

from activation_additions import (
    prompt_utils,
    utils,
    metrics,
    hook_utils
)
from activation_additions.prompt_utils import (
    ActivationAddition,
    pad_tokens_to_match_activation_additions,
    get_block_name,
)
utils.enable_ipython_reload()

# Disable gradients to save memory during inference
_ = torch.set_grad_enabled(False)

from typing import List, Union,Dict
import pandas as pd

In [116]:

from transformer_lens import HookedTransformer
from activation_additions import (
    prompt_utils,
    hook_utils,
    hyperparameter_search
)

In [108]:
# %%
# Load a model
MODEL = HookedTransformer.from_pretrained(model_name="gpt2-xl", device="cpu")
_ = MODEL.to("cuda:0")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-xl into HookedTransformer
Moving model to device:  cuda:0


In [68]:
hyperparameter_search.conditional_perplexity(MODEL,
    MODEL.to_tokens("The only thing, that we need to fear is"),
    MODEL.to_tokens(" fear itself")[:,1:]
) 

6.898438930511475

In [118]:
perplexity=hyperparameter_search.conditional_perplexity(MODEL,
    MODEL.to_tokens("The only thing we have to fear is"),MODEL.to_tokens(" fear itself")[:, 1:])
perplexity

0.4616594463586807

In [120]:
ActAds =[prompt_utils.ActivationAddition(coeff=4, act_name=6,prompt=" Pizza!"),
         prompt_utils.ActivationAddition(coeff=-4, act_name=6,prompt=" fear")]

In [121]:
hyperparameter_search.completion_perplexities(
    MODEL,
    [MODEL.to_tokens("The only thing we have to fear is")],
    [MODEL.to_tokens(" fear itself")[:, 1:]],
    [MODEL.to_tokens(" Pizza!")[:, 1:]],
    ActAds
)

([8.31489634513855], [5.5109670758247375])

In [124]:
df=hyperparameter_search.layer_coefficient_gridsearch(
    MODEL,
    ["The only thing we have to fear is", "I feel really"],
    {" Pizza!":1," fear":-1},
    [1,2,3,4],
    [3,6,9,12],
    [" fear itself", " afraid"],
    [" Pizza!", " hungry"]
)

In [125]:
df

,Layer,Coefficient,Perplexity (wanted),Perplexity (unwanted)
0,1,3,9.265073,7.755989
1,1,3,13.439856,10.905490
2,1,6,13.199443,8.092573
3,1,6,12.226157,10.160707
4,1,9,14.167190,8.620543
5,1,9,10.603421,9.711972
6,1,12,13.835853,9.299639
7,1,12,10.121796,9.510039
8,2,3,9.697480,7.860976
9,2,3,14.010214,11.077999


In [52]:
MODEL.remove_all_hook_fns()
add_actads_to_model(MODEL,ActAds)

In [48]:
ActAds =[prompt_utils.ActivationAddition(coeff=4, act_name=6,prompt=" hate"),
         prompt_utils.ActivationAddition(coeff=-4, act_name=6,prompt=" love")]
hook_fns=hook_utils.hook_fns_from_activation_additions(MODEL, ActAds)

In [49]:
for act_name in hook_fns.keys():
    for hook_fn in hook_fns[act_name]:
        MODEL.add_hook(act_name, hook_fn)

In [31]:
default_kwargs = {'temperature': 1, 'freq_penalty': 1, 'top_p': .3, 'model': MODEL}
get_x_vector_preset = partial(prompt_utils.get_x_vector, pad_method="tokens_right",
                              model=MODEL,
                              custom_pad_id=MODEL.to_single_token(" "))
steering_vector=[*get_x_vector_preset(prompt1="hello", prompt2="bye", coeff=2, act_name=2)]

In [32]:
hook_fns=hook_utils.hook_fns_from_activation_additions(MODEL, steering_vector)

In [27]:
#hook_fns =hook_utils.hook_fns_from_rich_prompts(model=MODEL, rich_prompts=steering_vector)

AttributeError: module 'activation_additions.hook_utils' has no attribute 'hook_fns_from_rich_prompts'

In [55]:
for act_name, hook_fn in hook_fns.items():
    print(type(act_name))
    print(act_name)
    print(type(hook_fn))
    print(hook_fn[1])
    #MODEL.add_hook(act_name, hook_fn[0])

<class 'str'>
blocks.6.hook_resid_pre
<class 'list'>
<function hook_fn_from_activations.<locals>.prompt_hook at 0x7f433c5a4ca0>


In [41]:
name="blocks.2.hook_resid_pre"
for hook_fn in hook_fns[name]:
    print(type(hook_fn))
    MODEL.add_hook(name, hook_fn)

<class 'function'>
<class 'function'>


In [42]:
tokens=MODEL.to_tokens("Hello workd")[0]

In [37]:
MODEL.remove_all_hook_fns()

In [53]:

gpt2_logits, gpt2_cache = MODEL.run_with_cache(tokens)


In [20]:
hook

NameError: name 'hook' is not defined